# Deploy Keras Model to the Cloud
1. Download data
2. Hot Encode Categorical Columns
3. Develop Model (use: ```RMSprop(lr=, rho=, epsilon=, decay=)```)
3. Create Storage Bucket (should already exist)
4. Authenticate on Google Cloud Platform
5. Deploy your model using the ```gcloud``` tool

### Download data

In [ ]:
!wget https://storage.googleapis.com/nicksdemobucket/bank.csv

--2020-11-21 20:06:44--  https://storage.googleapis.com/nicksdemobucket/bank.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.7.144, 172.217.12.240, 172.253.63.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.7.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4610348 (4.4M) [application/octet-stream]
Saving to: ‘bank.csv.7’

bank.csv.7          100%[===================>]   4.40M  --.-KB/s    in 0.02s   

2020-11-21 20:06:44 (265 MB/s) - ‘bank.csv.7’ saved [4610348/4610348]



In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
df = pd.read_csv('bank.csv', sep=";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [ ]:
df.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


### Process Data

In [ ]:
df.loc[df['y'] == 'no', 'y'] = 0
df.loc[df['y'] == 'yes', 'y'] = 1
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,0
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,0


In [ ]:
categorical = df.select_dtypes(include='object').columns
print(categorical)

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome', 'y'],
      dtype='object')


In [ ]:
frames = [pd.DataFrame(
  {
    'age': df['age'],
    'balance': df['balance'],
    'day': df['day'],
    'duration': df['duration'],
    'campaign': df['campaign'],
    'pdays': df['pdays'],
    'previous': df['previous'],
  }
)]


for column in categorical:
  onehot = pd.get_dummies(df[column], prefix=column)
  frames.append(pd.DataFrame(onehot))

onehot_X = pd.concat(frames, axis=1)

X = np.asarray(onehot_X.drop(['y_0', 'y_1'], axis=1).values).astype(np.int32)
y = np.asarray(df[['y']].values).astype(np.int32)

print('X:', X)
print('y:', y)

X: [[  58 2143    5 ...    0    0    1]
 [  44   29    5 ...    0    0    1]
 [  33    2    5 ...    0    0    1]
 ...
 [  72 5715   17 ...    0    1    0]
 [  57  668   17 ...    0    0    1]
 [  37 2971   17 ...    1    0    0]]
y: [[0]
 [0]
 [0]
 ...
 [1]
 [0]
 [0]]


In [ ]:
print('X.shape:', X.shape)
print('y.shape:', y.shape)

X.shape: (45211, 51)
y.shape: (45211, 1)


### Develop Model

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(
  learning_rate=0.001,
  rho=0.9,
  epsilon=1e-07,
  decay=0
)

In [ ]:
inputs = tf.keras.Input(shape=(51,))
dense_layer = tf.keras.layers.Dense(4, activation='relu')(inputs)
outputs = tf.keras.layers.Dense(1, activation='softmax')(dense_layer)

model = tf.keras.Model(inputs, outputs)
model.compile(
  loss='mean_squared_error',
  optimizer = optimizer,
  metrics=['mean_absolute_error', 'mean_squared_error']
)
model.summary()

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 51)]              0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 208       
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 5         
Total params: 213
Trainable params: 213
Non-trainable params: 0
_________________________________________________________________


In [ ]:
h = model.fit(
  X, y, 
  epochs=40,
  batch_size=4,
  steps_per_epoch=int(100/4), 
  validation_split=0.2
)

Epoch 1/40
25/25 [==============================] - 3s 134ms/step - loss: 0.9200 - mean_absolute_error: 0.9200 - mean_squared_error: 0.9200 - val_loss: 0.6842 - val_mean_absolute_error: 0.6842 - val_mean_squared_error: 0.6842
Epoch 2/40
25/25 [==============================] - 3s 130ms/step - loss: 0.9600 - mean_absolute_error: 0.9600 - mean_squared_error: 0.9600 - val_loss: 0.6842 - val_mean_absolute_error: 0.6842 - val_mean_squared_error: 0.6842
Epoch 3/40
25/25 [==============================] - 3s 132ms/step - loss: 0.9200 - mean_absolute_error: 0.9200 - mean_squared_error: 0.9200 - val_loss: 0.6842 - val_mean_absolute_error: 0.6842 - val_mean_squared_error: 0.6842
Epoch 4/40
25/25 [==============================] - 3s 139ms/step - loss: 0.9800 - mean_absolute_error: 0.9800 - mean_squared_error: 0.9800 - val_loss: 0.6842 - val_mean_absolute_error: 0.6842 - val_mean_squared_error: 0.6842
Epoch 5/40
25/25 [==============================] - 3s 140ms/step - loss: 0.8700 - mean_absolute

In [ ]:
h.history

{'loss': [0.9200000166893005,
  0.9599999785423279,
  0.9200000166893005,
  0.9800000190734863,
  0.8700000047683716,
  0.9800000190734863,
  0.9399999976158142,
  0.8799999952316284,
  0.8799999952316284,
  0.8999999761581421,
  0.9300000071525574,
  0.9399999976158142,
  0.9399999976158142,
  0.949999988079071,
  0.9700000286102295,
  0.9399999976158142,
  0.9100000262260437,
  0.9200000166893005,
  0.949999988079071,
  0.949999988079071,
  0.8999999761581421,
  0.9200000166893005,
  0.9300000071525574,
  0.9300000071525574,
  0.9200000166893005,
  0.9100000262260437,
  0.9399999976158142,
  0.8999999761581421,
  0.9200000166893005,
  0.949999988079071,
  0.9700000286102295,
  0.949999988079071,
  0.8799999952316284,
  0.8999999761581421,
  0.9399999976158142,
  0.9599999785423279,
  0.9399999976158142,
  0.9200000166893005,
  0.8700000047683716,
  0.9200000166893005],
 'mean_absolute_error': [0.9200000166893005,
  0.9599999785423279,
  0.9200000166893005,
  0.9800000190734863,
  0.8

### Set project name, bucket name, authenticate to GCP

In [ ]:
PROJECT_ID = "<INSERT_PROJECT_ID>"
BUCKET_NAME = "<INSERT_BUCKET_NAME>"
REGION = "<INSERT_REGION>"

! gcloud config set project $PROJECT_ID
! echo $PROJECT_ID

Updated property [core/project].
gcp-intro-1


In [ ]:
import sys

# if 'google.colab' in sys.modules:
from google.colab import auth as google_auth
google_auth.authenticate_user()
# else:
#   %env GOOGLE_APPLICATION_CREDENTIALS ''

In [ ]:
# authenticate_user didnt work for me, so i had to login
! gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=OErb06out8IS4mF2kuJnVECoZ9RXxo&prompt=consent&access_type=offline&code_challenge=68tRYocV2Ifan00vMEan4FDwL_jG_KbT3iBoGA78644&code_challenge_method=S256

Enter verification code: 4/1AY0e-g4UxlpkQnSzI4B08LMWvr9dEmsTLSe0-fADuwtho9JEvuVGya2XroY

You are now logged in as [nebiogludilan@gmail.com].
Your current project is [gcp-intro-1].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [ ]:
! gcloud projects list

PROJECT_ID              NAME         PROJECT_NUMBER
key-transformer-296313  gcp-intro-1  841478298350


In [ ]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


In [ ]:
# create bucket manually because it doesnt let you create it through cmd line
!gsutil mb -p $PROJECT_ID -l $REGION gs://$BUCKET_NAME
!gsutil ls -al gs://$BUCKET_NAME

Creating gs://dilo-test-ml-2/...
AccessDeniedException: 403 The project to be billed is associated with a closed billing account.


In [ ]:
JOB_DIR = 'gs://' + BUCKET_NAME + '/mcgilldemo2'

### Deploy Model

In [ ]:
export_path = tf.keras.models.save_model(model, JOB_DIR + '/keras_export')
print("Model exported to: ", export_path)

INFO:tensorflow:Assets written to: gs://dilo-test-ml-2/mcgilldemo2/keras_export/assets
Model exported to:  None


In [ ]:
MODEL_NAME = "<INSERT_MODEL_NAME>"

! gcloud ai-platform models create $MODEL_NAME --regions $REGION


Using endpoint [https://ml.googleapis.com/]
Created ml engine model [projects/key-transformer-296313/models/mcgilldemo2].


In [ ]:
MODEL_VERSION = "v1"
KERAS_EXPORT_DIRS = ! gsutil ls $JOB_DIR/keras_export/
SAVED_MODEL_PATH = KERAS_EXPORT_DIRS[0]

! gcloud ai-platform versions create $MODEL_VERSION \
  --model $MODEL_NAME \
  --runtime-version 1.13 \
  --python-version 3.5 \
  --framework tensorflow \
  --origin $SAVED_MODEL_PATH

Using endpoint [https://ml.googleapis.com/]


### Test Model

In [ ]:
X[0]

array([  58, 2143,    5,  261,    1,   -1,    0,    0,    0,    0,    0,
          1,    0,    0,    0,    0,    0,    0,    0,    0,    1,    0,
          0,    0,    1,    0,    1,    0,    0,    1,    1,    0,    0,
          0,    1,    0,    0,    0,    0,    0,    0,    0,    0,    1,
          0,    0,    0,    0,    0,    0,    1], dtype=int32)

In [ ]:
%%bash
rm prediction_input.json
touch prediction_input.json
echo "[58, 2143, 5, 261, 1, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]" >> prediction_input.json

cat prediction_input.json

[58, 2143, 5, 261, 1, -1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1]


In [ ]:
! gcloud ai-platform predict \
  --model $MODEL_NAME \
  --version $MODEL_VERSION \
  --json-instances prediction_input.json

Using endpoint [https://ml.googleapis.com/]
DENSE_9
[1.0]


In [ ]:
y[0]

array([0], dtype=int32)

### DELETE

In [ ]:
# TO DELETE BUCKET/model
! gcloud ai-platform versions delete $MODEL_VERSION --quiet --model $MODEL_NAME
! gcloud ai-platform models delete $MODEL_NAME --quiet
! gsutil rm -r gs://$BUCKET_NAME

Using endpoint [https://ml.googleapis.com/]
Using endpoint [https://ml.googleapis.com/]
Removing gs://dilo-test-ml-2/mcgilldemo/#1605988802511673...
Removing gs://dilo-test-ml-2/mcgilldemo/keras_export/#1605988803111147...
Removing gs://dilo-test-ml-2/mcgilldemo/keras_export/assets/#1605988809010325...
Removing gs://dilo-test-ml-2/mcgilldemo/keras_export/saved_model.pb#1605988889412132...
/ [4 objects]                                                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m rm ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Removing gs://dilo-test-ml-2/mcgilldemo/keras_export/variables/#1605988803710531...
Removing gs://dilo-test-ml-2/mcgilldemo/keras_export/variables/variables.data-00000-of-00001#1605988888272842...
Removing gs://dilo-test-ml-2/mcgilldemo/keras_export/variables/variables.index